In [ ]:
import feedparser
from newspaper import Article
import html

"""
Every minute;
1. Fetch feed for all RSS URLs. Filter out feed data for last 30 min. - done
2. Maintain a cache with TTL of 60 min. If the current feed title does not exist in the cache, push it and add it to processinglist
4. For each of this title, analyse the news

"""


def get_summary(url):
    article = Article(url)
    article.download()
    article.parse()
    #print("Authors:", article.authors)
    return article.text

def get_articles(url):
    feed = feedparser.parse(url)
    for entry in feed.entries:
        title = entry.title
        #description = entry.summary
        published_date = entry.published
        text = get_summary(entry.links[0]['href'])
        print(f"{published_date} - {html.unescape(title)}")
        #print(f"Published: {published_date}")
        #print(f"Text: {text}")
        print("----------------------------------------------------------------------------------------------------")

get_articles('https://www.cnbctv18.com/commonfeeds/v1/cne/rss/economy.xml')
# get_articles('https://rss.app/feeds/tgll9PXJFaUoCjGV.xml')

In [ ]:
import time

feed = feedparser.parse('https://www.cnbctv18.com/commonfeeds/v1/cne/rss/economy.xml')
feed = feed.entries

current_timestamp = time.mktime(time.gmtime())
window = 60

for i in range(len(feed)-1 , -1, -1):
    print(feed[i])
    published_ts = time.mktime(feed[i].get('published_parsed'))
    if current_timestamp - published_ts > window:
        del feed[i]

print(feed)

In [ ]:
import requests
import xml.etree.ElementTree as ET

url = "https://in.investing.com/news/stock-market-news/market-outlook-rbi-mpc-q2-results-iip-data-key-factors-to-watch-next-week-4459037"

payload = ""
headers = {
    "User-Agent": "insomnium/0.2.3-a"
}

response = requests.request("GET", url, headers=headers)

#print(response.text)

article = Article('')
article.set_html(response.text)
article.html

In [ ]:
# Parse the article
article.parse()

# Now you can access the parsed content
print("Title:", article.title)
print("Text:", article.text)
print("Authors:", article.authors)
print("Published Date:", article.publish_date)

In [ ]:
response = requests.request(
    method="GET",
    headers={
        "User-Agent": "insomnium/0.2.3-a"
    },
    url="https://www.business-standard.com/rss/markets-106.rss"
)

if not 200 <= response.status_code < 400:
    raise Exception (f"Failed to fetch feed from RSS {url}")

feedData = feedparser.parse(response.text)

In [ ]:
article.html

In [ ]:
from bs4 import BeautifulSoup
import json
import requests
import xml.etree.ElementTree as ET

url = "https://in.investing.com/news/stock-market-news/market-outlook-rbi-mpc-q2-results-iip-data-key-factors-to-watch-next-week-4459037"

payload = ""
headers = {
    "User-Agent": "insomnium/0.2.3-a"
}

response = requests.request("GET", url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

# Example: Extract the text from all <p> tags (adjust according to the HTML structure)

# Find the specific <script> tag containing the desired data
# For example, assuming the data is embedded in a script tag containing "window.articleData" or similar
script_tags = soup.find_all('script', type='application/ld+json')

# Initialize a variable to hold the article body
article_body = None

# Loop through all script tags and look for the NewsArticle
for script_tag in script_tags:
    try:
        # Parse the JSON content
        data = json.loads(script_tag.string)
        
        # Check if it's of type NewsArticle and extract the articleBody
        if data.get('@type') == 'NewsArticle':
            article_body = data.get('articleBody')
            break  # Exit loop after finding the first NewsArticle
    except json.JSONDecodeError:
        continue  # Skip any script that does not contain valid JSON

# Print the extracted article body
if article_body:
    print("Article Body:", article_body)
else:
    print("Article Body not found.")

In [ ]:
response.content

In [ ]:
requests.request("GET","https://b2b.tijorifinance.com/b2b/v1/in/kite-widget/web/equity/TRIDENT/?exchange=NSE&broker=kite&theme=dark"
                ,headers={
                    "User-Agent" : "insomnium/0.2.3-a"
                }
                ).text